# 📘 Project Overview: English → French Machine Translation

## 1. Introduction

In this project, I explored transformer-based models for machine translation.
The task focuses on English-to-French translation using a large-scale parallel dataset.

The original dataset contains around 20 million sentence pairs, which makes it very rich in linguistic context and phrasing. However, since this is my first practical experiment with transformer-based translation models, and due to computational and resource limitations, a subset of 100,000 samples was randomly selected and used for this project.

**The main goal of this project is to:**
- Understand the full workflow of transformer-based translation.
- Experiment with fine-tuning a pre-trained model.
- Evaluate translation quality using standard metrics.

## 2. Dataset Description

- **Dataset**: English-to-French Translation Dataset
- **Languages**: English (source) → French (target)
- **Original size**: Over 22.5 million samples
- **Used subset**: 100,000 samples

## 3. Project Workflow

The project follows the standard transformer-based NLP pipeline, structured as follows:

- Data Loading
- Data Cleaning
- Data Preprocessing
    - Tokenization
    - Data Collation
    - Trainer setup
- Model Fine-Tuning
- Model Evaluation (BLEU Score)
- Testing on Sample Translations

## 4. Data Cleaning

During the data cleaning phase, the dataset was first explored to understand its structure and quality. This included:

- Inspecting the dataset shape and schema
- Checking for missing values
- Identifying duplicate entries

**The following steps were applied:**
- Dropping rows with missing values
- Analyzing the minimum and maximum sentence lengths for both English and French columns
- Filtering out sentences with extremely short or excessively long lengths to reduce noise and improve model stability

## 5. Data Preprocessing

In the preprocessing stage, several steps were performed to prepare the data for the transformer model:

**Text normalization:**
- Converted text to lowercase

**Dataset splitting:**
- Training set
- Validation (evaluation) set
- Testing set

**Tokenization:**
- Used AutoTokenizer, noting that each transformer model has its own tokenizer
- Defined model inputs (English text) and labels (French text)

**Data Collator:**
- Applied padding to unify sequence lengths within each batch
- Generated attention masks automatically to identify which is the real text and which is the padding one
- Prepared batches efficiently for training

**Trainer Pipeline:**
- Used the Hugging Face Trainer API to streamline training
- Integrated the model, tokenizer, data collator, and training arguments into a single pipeline

## 6. Model Fine-Tuning

For the translation task, the pre-trained model Helsinki-NLP/opus-mt-en-fr was fine-tuned.

**Training details:**
- Number of epochs: 3
- Optimized using several training arguments (learning rate, batch size, evaluation strategy, etc.)

## 7. Model Evaluation

The model was evaluated using the **BLEU Score**, a standard metric for machine translation quality.

- Evaluation set: Testing dataset
- BLEU Score achieved: 58.69%

A BLEU score of 58.69% is considered a strong and promising result for a neural machine translation task, especially given the limited training data and computational constraints.

## 8. Testing and Inference

Finally, the fine-tuned model was tested on several samples from the test set to:
- Visually inspect translation quality
- Compare predicted translations with ground-truth French sentences
- Validate the practical performance of the model

In [1]:
pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Data Loading

In [2]:
# data loading
import pandas as pd
def load_data(filepath,nrows = None ):
    df = pd.read_csv(filepath, nrows=nrows)
    return df
    

# Data Cleaning

In [3]:
# Text cleaning
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# dataset exploration
def explore_data(df):
    data_shape = df.shape
    data_info = df.info()
    missing_vals = df.isnull().sum()
    duplicated_vals = df.duplicated().sum()
    print('Data Shape: ', data_shape)
    print('Data Info: ', data_info)
    print('Missing Values: ',missing_vals )
    print('duplicated Values: ', duplicated_vals)

# data cleaning
def data_cleaning(df):
    df = df.dropna()
    print ('Text Statistics')
    print('English Language')
    print(f"Average length: {df['en'].astype(str).str.len().mean():.1f} characters")
    print(f"Min length: {df['en'].astype(str).str.len().min()} characters")
    print(f"Max length: {df['en'].astype(str).str.len().max()} characters")
    
    print('\nFrench Language')
    print(f"Average length: {df['fr'].astype(str).str.len().mean():.1f} characters")
    print(f"Min length: {df['fr'].astype(str).str.len().min()} characters")
    print(f"Max length: {df['fr'].astype(str).str.len().max()} characters")
    return df

def text_cleaning(df):
    # Length filtering 
    df = df[
        (df['en'].str.split().str.len() >= 1) & 
        (df['en'].str.split().str.len() <= 128) &
        (df['fr'].str.split().str.len() >= 1) & 
        (df['fr'].str.split().str.len() <= 128)
    ]
    # Normalize: lowercase, strip whitespace
    df['en'] = df['en'].str.lower().str.strip()
    df['fr'] = df['fr'].str.lower().str.strip()
    return df

# data splitting
def data_splitting(df):
    temp_df, test_df = train_test_split(df, test_size = 0.15, random_state = 42)
    train_df, val_df = train_test_split(temp_df, test_size = 0.175, random_state = 42)
    print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    # Convert to Hugging Face Datasets
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)
    return train_dataset, val_dataset, test_dataset

# Data Preprocessing - Tokenization

In [4]:
# data preprocessing (tokenizer)
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

def preprocessing(examples):
    inputs = examples['en']
    targets = examples['fr']
    
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding='longest',
        truncation=True
    )
    
    labels = tokenizer(
        targets, 
        max_length=512,
        padding='longest',
        truncation=True
    )
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing to datasets (batched for speed)
def tokenize_datasets(train_dataset, val_dataset, test_dataset):
    tokenized_train = train_dataset.map(preprocessing, batched=True, batch_size=1000)
    tokenized_val = val_dataset.map(preprocessing, batched=True, batch_size=1000)
    tokenized_test = test_dataset.map(preprocessing, batched=True, batch_size=1000)
    
    print(f"Tokenized train: {len(tokenized_train)} samples")
    print(f"Tokenized val: {len(tokenized_val)} samples")
    print(f"Tokenized test: {len(tokenized_test)} samples")
    
    return tokenized_train, tokenized_val, tokenized_test

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


# Model Setup & Fine-Tuning

In [5]:
# Model setup and data collator
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
import os

def setup_model_and_collator(tokenizer):
    # Disable wandb if causing issues
    os.environ["WANDB_DISABLED"] = "true"
    
    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
    
    # Move to GPU if available
    if torch.cuda.is_available():
        model = model.cuda()
        print("Model moved to GPU")
    else:
        print("Using CPU")
        
    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True
    )
    
    print("Model and data collator ready!")
    
    return model, data_collator

2026-01-26 12:59:13.898109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769432354.105076      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769432354.161851      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769432354.626495      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769432354.626537      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769432354.626540      24 computation_placer.cc:177] computation placer alr

In [6]:
# Training setup and execution
from transformers import Trainer, TrainingArguments
import torch

def setup_training_args(output_dir='/kaggle/working/results', 
                        num_epochs=3,
                        train_batch_size=4,
                        eval_batch_size=4,
                        learning_rate=3e-5):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=eval_batch_size,
        gradient_accumulation_steps=16,
        eval_strategy='epoch',
        save_strategy='epoch',
        logging_dir='/kaggle/working/logs',
        logging_steps=500,
        fp16=True if torch.cuda.is_available() else False,
        learning_rate=learning_rate,
        report_to="none",
        gradient_checkpointing=True,
    )
    return training_args

def train_model(model, training_args, tokenized_train, tokenized_val, data_collator, 
                save_dir='/kaggle/working/fine_tuned_marian'):
    # Create Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        data_collator=data_collator,
    )
    
    print("Starting training...")
    # Train
    trainer.train()
    
    print(f"Training complete! Saving model to {save_dir}")
    # Save model
    trainer.save_model(save_dir)
    
    return trainer

# Model Evaluation

In [7]:
# Evaluation
import torch
from sacrebleu.metrics import BLEU
from tqdm import tqdm


def evaluate_model(model, tokenizer, test_dataset):

    # Set model to evaluation mode
    model.eval()
    
    # Generate predictions
    predictions = []
    references = []
    
    print(f"Generating predictions for entire test set ({len(test_dataset)} samples)...")
    
    with torch.no_grad():  # Disable gradient calculations for speed
        for example in tqdm(test_dataset, desc="Evaluating"):
            inputs = tokenizer(
                example['en'],
                return_tensors='pt',
                padding=True,
                truncation=True,
                max_length=512
            ).to(model.device)
            
            # Generate translation
            outputs = model.generate(
                **inputs,
                max_length=512,
                num_beams=4,  # Beam search for better quality
                early_stopping=True
            )
            
            # Decode prediction
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predictions.append(pred)
            references.append([example['fr']])  # sacrebleu expects list of references
    
    # Compute BLEU score
    bleu = BLEU()
    score = bleu.corpus_score(predictions, references)
    
    print(f"\nBLEU Score: {score.score:.2f}")
    print(f"Evaluated {len(predictions)} translations")
    
    return score, predictions, references

# Model Testing 

In [8]:
# Inference
import torch

def show_translation_examples(model, tokenizer, test_dataset, num_examples=5):
    model.eval()
    
    print("\nTranslation Examples:")
    print("=" * 60)
    
    for i in range(min(num_examples, len(test_dataset))):
        ex = test_dataset[i]
        
        # Prepare input
        inp = tokenizer(ex['en'], return_tensors='pt', truncation=True).to(model.device)
        
        # Generate translation
        with torch.no_grad():
            out = model.generate(**inp, max_length=512, num_beams=4)
        
        # Decode prediction
        pred = tokenizer.decode(out[0], skip_special_tokens=True)
        
        print(f"\nExample {i+1}:")
        print(f"{'='*60}")
        print(f"EN:        {ex['en']}")
        print(f"REF (FR):  {ex['fr']}")
        print(f"PRED (FR): {pred}")
    
    print(f"\n{'='*60}")
    print(f"Displayed {min(num_examples, len(test_dataset))} translation examples")


def translate_text(model, tokenizer, text):
    model.eval()
    
    # Prepare input
    inp = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(model.device)
    
    # Generate translation
    with torch.no_grad():
        out = model.generate(**inp, max_length=512, num_beams=4, early_stopping=True)
    
    # Decode and return
    translation = tokenizer.decode(out[0], skip_special_tokens=True)
    
    return translation

# Main Pipeline

In [9]:
# Main function to run the entire pipeline
def main():
    print("="*60)
    print("Translation Model Training Pipeline")
    print("="*60)
    
    # Step 1: Load data
    print("\nLoading data...")
    df_path = '/kaggle/input/en-fr-translation-dataset/en-fr.csv'
    df = load_data(df_path, nrows=100000)
    print(f"Loaded {len(df)} rows")
    print(df.head())
    
    # Step 2: Explore data
    print("\nExploring data...")
    explore_data(df)
    
    # Step 3: Clean data
    print("\nCleaning data...")
    df = data_cleaning(df)
    
    # Step 4: Normalize text
    print("\nNormalizing text...")
    df = text_cleaning(df)
    
    # Step 5: Split data
    print("\nSplitting data...")
    train_dataset, val_dataset, test_dataset = data_splitting(df)
    
    # Step 6: Tokenize datasets
    print("\nTokenizing datasets...")
    tokenized_train, tokenized_val, tokenized_test = tokenize_datasets(
        train_dataset, val_dataset, test_dataset
    )
    
    # Step 7: Setup model and collator
    print("\nSetting up model and data collator...")
    model, data_collator = setup_model_and_collator(tokenizer)
    
    # Step 8: Setup training and train
    print("\nTraining model...")
    training_args = setup_training_args(
        output_dir='/kaggle/working/results',
        num_epochs=3,
        train_batch_size=4,
        eval_batch_size=4,
        learning_rate=3e-5
    )
    
    trainer = train_model(
        model=model,
        training_args=training_args,
        tokenized_train=tokenized_train,
        tokenized_val=tokenized_val,
        data_collator=data_collator,
        save_dir='/kaggle/working/fine_tuned_marian'
    )
    
    # Evaluation
    print("\n" + "="*60)
    print("EVALUATION")
    print("="*60)
    score, predictions, references = evaluate_model(
        model=model,
        tokenizer=tokenizer,
        test_dataset=test_dataset
    )
    
    # Show examples
    print("\n" + "="*60)
    print("Translation Examples")
    print("="*60)
    show_translation_examples(
        model=model,
        tokenizer=tokenizer,
        test_dataset=test_dataset,
        num_examples=5
    )
    
    print("\n" + "="*60)
    print("Pipeline Complete!")
    print("="*60)
    print(f"Final BLEU Score: {score.score:.2f}")
    print(f"Model saved to: /kaggle/working/fine_tuned_marian")


if __name__ == '__main__':
    main()

Translation Model Training Pipeline

Loading data...
Loaded 100000 rows
                                                  en  \
0  Changing Lives | Changing Society | How It Wor...   
1                                           Site map   
2                                           Feedback   
3                                            Credits   
4                                           Français   

                                                  fr  
0  Il a transformé notre vie | Il a transformé la...  
1                                       Plan du site  
2                                        Rétroaction  
3                                            Crédits  
4                                            English  

Exploring data...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   en      99998 non-null   object
 1   fr      100000 no

/tmp/ipykernel_24/1211071959.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['en'] = df['en'].str.lower().str.strip()
/tmp/ipykernel_24/1211071959.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fr'] = df['fr'].str.lower().str.strip()



Splitting data...
Train: 69864, Val: 14820, Test: 14945

Tokenizing datasets...


Map:   0%|          | 0/69864 [00:00<?, ? examples/s]

Map:   0%|          | 0/14820 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss
1,0.256000,0.216977
2,0.209800,0.195082
3,0.194400,0.190283


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training complete! Saving model to /kaggle/working/fine_tuned_marian

EVALUATION
Generating predictions for entire test set (14945 samples)...


Evaluating: 100%|██████████| 14945/14945 [1:51:35<00:00,  2.23it/s]



BLEU Score: 58.69
Evaluated 14945 translations

Translation Examples

Translation Examples:

Example 1:
EN:        next, students will use the atlas’ interactive map of terrestrial ecozones map to collect additional information about the land cover regions.
REF (FR):  ensuite, ils se servent de la carte interactive relative aux écozones terrestres, qui se trouve dans l’atlas, afin de recueillir de l’information additionnelle au sujet des régions de couverture des terres.
PRED (FR): ensuite, les élèves utiliseront la carte interactive atlas de la carte des écozones terrestres pour recueillir des renseignements supplémentaires au sujet des régions de couverture des terres.

Example 2:
EN:        uni is a traditional product in japan and presenting it to quality conscious japanese consumers as a differentiated product requires a completely honest appraisal process to retain that most valuable of supplier attributes, credibility.
REF (FR):  l’œuf d’oursin est un produit traditionnel au ja